# HW7: Counting and regular expressions



In [1]:
# re module for working with regular expressions
import re
# For numerical work, nearly everyone uses numpy
from numpy import pi

## Part 1: Dictionaries and counting

This notebook partly draws from materials put together by [Dirk Hovy](http://dirkhovy.com/). That's why there's a figure today! Dirk is a computational linguist at the University of Copenhagen. Much of his work tries to explore the intersection of social variables and NLP, working with large online corpora.

### The structure of programs

Most of programming, irrespective of the language you use, has four main elements:

1. ***Assignment***: linking a name to a value. The names are called ***variables***. 

2. ***Loops***: sometimes we want to do the same thing repeatedly, either a fixed number of times, or
until something happens. This is what loops are for. 

3. ***I/O (Input/Output)***: this refers to everything that has to do with getting information into and
out of our programs, e.g. files (opening, closing, reading from or writing to them) or output on
the screen.

4. ***Control structures***: sometimes, we need to make decisions. I.e., if a variable has a certain 
value, do `X`, otherwise, do `Y`. Control structures are simple `if...then...else` constructs that evaluate
the alternatives and make this decision. 

Today we'll put these together to do a useful elementary language processing task: getting counts of words in a document. The three main new things we need to learn today are: **reading from files**, **control structures**, and an important new data type the **dictionary** or just **dict**, which is a **mapping** data structure.

### The dictionary (or "dict") data type

Python uses the term "dictionary" or "dict" for a *mapping*: a collection of items of one type mapping to another type. A dictionary is written with curly braces. For example, here's a mapping, from web sites to my passwords:

In [2]:
passwds = {'Amazon': 'curly', 'Google': 'furry', 'Apple': 'easy',
           'Microsoft' : 'easy'}

No, not really! But it will do. You can access elements from a dict using the same square brackets notation after the dict/variable name, but now using a key which is the first half of the mapping:

In [3]:
print('My Google password is: ' + passwds['Google'])

My Google password is: furry


Trying to get a value for a key that doesn't exist is an error!

In [4]:
#passwds['LinkedIN']

If we want to add a new item to our dictionary, we can simply assign a key a value:
```
<dictionary>[key] = <value>
```

Add the value `"flotilla"` as my `"Facebook"` pasword:

In [5]:
# Add the value "flotilla" as my "Facebook" pasword:

passwds['Facebook'] = 'flotilla'

passwds

{'Amazon': 'curly',
 'Google': 'furry',
 'Apple': 'easy',
 'Microsoft': 'easy',
 'Facebook': 'flotilla'}

In a dict, there can only be one value for a key, but several keys can have the same value. Oh, and while I said a key can have only one value, that value _can_ be a list, which lets you do general relations. A dictionary, unlike a list, isn't ordered. But you can very efficiently get the value for a key. You can also call 3 method `keys()`, `values()`, and `items()` which return list-like values that you can do a `for`-loop over to see all the keys, values, and mappings in the dict.  Try them:

In [6]:
for k in passwds.keys():
    print(k)

Amazon
Google
Apple
Microsoft
Facebook


Note that the keys didn't come out in the order that I wrote them down. You shouldn't rely on the order you wrote things down in.

In [7]:
# Now print all the values

passwds.values()

for k in passwds.values():
    print(k)

curly
furry
easy
easy
flotilla


In [8]:
# Now print all the items

passwds.items()

for k in passwds.items():
    print(k)

('Amazon', 'curly')
('Google', 'furry')
('Apple', 'easy')
('Microsoft', 'easy')
('Facebook', 'flotilla')


Note that the item is something we havent quite seen before – it's two strings wrapped in parentheses. It looks like the arguments to a function. This is different from a list and is called a ***tuple***. It's less important than lists, but we'll come back to them later today....

You can check whether a key or value is in a map with the `in` and `not in` operators: `<key> in <dict>`. But that's often tedious to use, so you should also know the cleverer method on dicts `get(key, default)`, which lets you ask for a key, and return its value if it exists, or the default value otherwise. We'll be able to use it later to make our program neater.

In [9]:
# See if I have an 'Amazon' password
'Amazon' in passwds

passwds.get('Amazon')

'curly'

In [10]:
# Either print my 'Facebook' password or 'None'
passwds.get('Facebook', 'None')

'flotilla'

### Word Counts—Dictionaries and Control Structures

Last week, we learned about variable assignment, loops, and printing to the screen.
There are several useful object types that we have not yet covered, and we need to learn about the constructs that let us
test conditions. We will see them in this program, as well as IO for reading from files.

We want to know which words occur how often in a
file. This is a common elementary text processing step in order to get some idea of your data and to get a sense of its overall topics. The output of such counting is precisely what people use to draw the very common visualization of [word clouds](http://www.wordle.net/). (Even though they're very common, many visualization people don't like them very much; just like pie charts.)

Let’s first think about what we have and what we want. We have a ***file***, and we want the
counts for the ***words*** in there. So there is a ***file***, ***sentences***, ***words***, and their ***counts***. We need to read the
file, get the sentences; for each sentence, get the words, and somehow record their counts. In the end,
we just print out the counts again. We can display this like in Figure 1.

<img src="pics/diagram_word_counts.png" width="500px">
<div align="center">*Figure 1: Flow chart for our word count problem*</div>



Now let’s look at the program: it takes a file, reads it in, keeps a running count for each word, and
prints those counts at the end. 

**Make sure to execute each code section as you progress (even the pre-written ones), so that the variables become available to the interpreter.** You won't see any direct output when executing the cell below, but we will need it further on.

We first declare the name of the file as a variable, and then actually open the file. 
`open()` is the function that reads in the file. It takes just one argument: the name of the file we
try to open. You can give it a second argument, `'w'` if you want to write to a file, rather than just read from it. Here, we only want to read, so we don't need to specify anything else.

Python takes care of some pesky new line and encoding issues, so we won’t worry too much right now about special characters. Go ahead and read a file with runes!

In [11]:
file_name = 'debate-clinton.txt'

# open the file for reading
text_file = open(file_name)


The result of running `open()` is not the text of the file. It is similar to a list (it's not exactly a list, but an ***iterator*** – that's also what `keys()` gave us above), and we call that list `text_file`, so we can use it later on. This give us a ***handle*** to read through the file. 

In [12]:
word_count = {}

After we have assigned the file hande, we assign the name `word_count` to a ***dictionary***. Here, our keys will be strings (the words), and the values we map them to are numbers (their respective
frequencies). If we just use a pair of curly braces, as we
did here, we get an empty dictionary. There are no entries. 

After we have declared the dictionary, we start iterating through the file with a `for`-loop.
Since `text_file` is an open file, this gives us a list of all the lines. We can
thus iterate over them. For each line in the file, we want to do a number of things.
That is why the next lines are all indented under the `for`-loop header line. 

In [13]:
# go through the open file line by line
for line in text_file:
    # get rid of the line break at the end
    line = line.strip()
    # split sentence along spaces
    sentence = line.split()
    # go through words
    for word in sentence:
        # check whether word is already in dictionary
        if word in word_count:
            # if yes: increment
            word_count[word] = word_count[word] + 1
        # if not, add an entry
        else:
            word_count[word] = 1

In [14]:
list(word_count.items())[:10]

[('How', 4),
 ('are', 34),
 ('you,', 3),
 ('Donald?', 1),
 ('Well,', 26),
 ('thank', 1),
 ('Lester,', 3),
 ('and', 113),
 ('thanks', 1),
 ('to', 220)]

First, we get rid of the line break and any white space at the beginning or end of the line. We could write code to do all these things, but there is an easier (and shorter) way:
we use the `strip()` command to remove all that whitespace from the line and assign it to 
the same name as before (`line`). (*Subtle point here about line break character!*) We made a new object – strings cannot be changed – but we assign it the same name. Whenever we use `line` from now on, it is the “cleaned-up” version of the line. (*Subtle points:* (1) Mutable and immutable objects; (2) If a method returns a new, changed object but doesn't change the original object – this is common and the only way to do things for immutable objects – then it is vital to assign the output of the method to something, or you will lose it. Commonly we assign it back to the same variable name if we conceptually think that we have *improved* the same thing.) 

We then use the `split()` command we have seen before next.
Remember, it splits a sentence at the white space subsequences into a list, so if we had extra white spaces,
it would create empty entries in our list. The list of strings resulting from `split()` is assigned to
`sentence`, and we then iterate over that list. We have seen this before, so I will skip to the next
interesting part here: control structures.

After we have read the whole file, we close it with the `close()` command on the file variable. The dot tells us that it is a property of files. Note 
that this line is no longer indented under the `for`-loop, but at the same level. This means that it is only
executed once we have completed all our iterations of the for-loop, in this case, after we have read all
lines in the file!

In [15]:
# close the file after reading
text_file.close()

### Control Structures

So far, we have simply executed one command after the next. We never had to make a decision or
choose among options. Now we have to. If a word is already listed in our dictionary, we want to
increase its count by one (we know how to do that). If the word is not in our dictionary, however, we
have to make an entry. Otherwise, we would try to increment a count that does not even exist (you
cannot look up something that is not in the dictionary).

To make the decision what to do, we use the `if...then...else` structure or ***conditional***.
The structure of the conditional is simple:
```
if <condition is true>:
    <action1>
else:
    <action2>
```

Here `<condition>` is another type of variable, a so-called ***boolean***. They are named after the 
mathematician Boole, and have only two values: `True` and `False` (note the capital spelling!). In our
case, the value comes from the outcome of the condition `word in word_count` to check whether
the dictionary `word_count` contains the key word. **`in`** is one of Python’s reserved words. You can
use it to check whether a variable is in a dictionary, a list, or other ***collections***.
Sometimes, there are more than just two cases (something being true or false) that we would
like to account for. In that case, we can check for more conditions:
```
if <condition1 is true>:
    <action1>
elif <condition2 is true>:
    <action2>
elif <condition3 is true>:
    <action3>
else:
    <action4>
```

You can add as many `elif` cases as you want! We will see an example of this in the next section.

So if the word we look at is indeed in our dictionary, we increment its count by one. 
This puts the current word in the dictionary and sets its counter to 1.

Write your own control structure that checks whether `"Amazon"` is a key in `passwds`, and prints `"Your password is <passwd>"` if there is one and `"You don't have an Amazon account!"` otherwise.

In [16]:
if 'Amazon' in passwds:
    print(passwds['Amazon'])
else:
    print("You don't have an Amazon account!")

curly


In our program, we finally want to print all the counts we have collected to the screen. We use another `for`-loop. This time, it iterates over a list of ***tuples***. Tuples are a lot like lists, with the big difference that they 
have a fixed size. They are less flexible than lists. In Python, we denote tuples by round brackets
(instead of square ones as for lists). The function `items()` of a dictionary returns a list of 
tuples of each key and its respective value. We use that and assign them to `word` and `frequency`, respectively. We print each word and its frequency (provided it occurred more than once) separated by a space, (that is why there is a comma in the `print()` statement, see above).

In [17]:
# take each pair of word and frequency in the dictionary
for (word, frequency) in word_count.items():
    if frequency > 1:
        print(word, frequency)

How 4
are 34
you, 3
Well, 26
Lester, 3
and 113
to 220
for 37
us. 2
The 6
question 4
in 90
this 26
is 51
really 18
what 37
kind 10
of 122
country 8
we 104
want 18
be 54
future 3
we'll 5
build 3
together. 2
my 9
second 3
so 19
I 133
think 36
about 33
a 105
First, 2
have 81
an 11
economy 5
that 108
works 2
everyone, 2
not 34
just 14
those 7
at 33
the 228
top. 3
That 12
means 4
need 23
new 9
jobs, 5
good 13
with 35
rising 3
incomes. 2
us 15
invest 4
you. 5
your 23
jobs 9
small 3
business, 3
because 20
most 4
will 19
come 5
from 17
business. 2
We 24
also 12
make 16
raising 2
national 4
minimum 2
equal 2
pay 7
work. 4
see 9
more 21
companies 2
do 32
If 3
you 50
help 4
create 4
should 15
able 6
share 3
them, 2
And 69
support 7
people 25
who 19
balance 2
family 3
I've 11
heard 4
many 10
difficult 2
face 3
you're 6
So 20
let's 6
paid 6
sure 5
care 2
debt-free 2
going 24
We're 6
it 43
by 11
having 5
wealthy 5
their 19
fair 5
close 2
corporate 2
on 25
Donald 15
Trump 2
Donald, 3
it's 13
where 7
i

Well, we've learned a fair bit here. We have learned how to read in a text file, how to use control structures, and we have
seen the new object types dictionaries and tuples.
You have now seen a lot of the basics of Python! While there are a lot of other things that you *can* learn – and, gosh, I'm going to attempt to teach quite a few of them — you can actually write quite a bit of basic text processing using just these elements. Many of the things that we'll learn later provide faster, more powerful, more convenient ways to do things that you _could_ do with just these elements.

### Doing more with word counts from the Clinton-Trump debate

Try to do all of these things, and end up with a decent program at the end that does all this stuff.

The above program got word counts from the file `'debate-clinton.txt'`. It was hardcoded to do so. But we also want word counts from `'debate-trump.txt'`. So, what we want is a function that can count words in _any_ file.  We might structure our program as two functions:

1. A function that takes a string filename and returns a dict from words to their counts in the file.

2. A function that takes a dict of word counts and prints the word counts

You are welcome to copy and paste any code from above to get this to work.

In [18]:
# Function for collecting word counts from a file
# Don't forget to close the file after you have read it in!

def word_counter(file_name): 
    text_file = open(file_name) 
    word_count = {}

    for line in text_file:
        sentence = line.strip().split() # doing it in one line now bc conciseness, however less readable 
        # go through words
        for word in sentence:
            if word in word_count:
                word_count[word] = word_count[word] + 1
            else:
                word_count[word] = 1
    
    text_file.close()
    
    return word_count


In [19]:
# Function for printing word counts from a dict

def print_wordcounts(word_count_dict):
    #Q: notice the name change – word_count_dict.items() – compare it to the previous code, why the change?
    for (word, frequency) in word_count_dict.items():
        if frequency > 5: #changed it to 5 so there is less output
            print(word, frequency)

In [20]:
# Top-level code that calls the above functions for each of the files
# 'debate-clinton.txt' and 'debate-trump.txt'.
# You probably also want to print out a blank line separator dividing the files and saying which you 
# are printing.

files = ['debate-clinton.txt', 'debate-trump.txt']

for file in files:
    # Q: why do I call file here when running the function when it is called 'file_name' in the function?
    word_count = word_counter(file)
    print("-----------------")
    print(f"currently printing the word count of file: {file}") #notice f-string here but not on line above
    print_wordcounts(word_count)

-----------------
currently printing the word count of file: debate-clinton.txt
are 34
Well, 26
and 113
to 220
for 37
The 6
in 90
this 26
is 51
really 18
what 37
kind 10
of 122
country 8
we 104
want 18
be 54
my 9
so 19
I 133
think 36
about 33
a 105
have 81
an 11
that 108
not 34
just 14
those 7
at 33
the 228
That 12
need 23
new 9
good 13
with 35
us 15
your 23
jobs 9
because 20
will 19
from 17
We 24
also 12
make 16
pay 7
see 9
more 21
do 32
you 50
should 15
able 6
And 69
support 7
people 25
who 19
I've 11
many 10
you're 6
So 20
let's 6
paid 6
going 24
We're 6
it 43
by 11
their 19
on 25
Donald 15
it's 13
where 7
important 7
our 38
country. 6
You 13
can 28
put 8
into 9
trade 7
other 6
tax 15
work 10
has 29
would 35
all 11
over 8
we've 13
ever 7
that's 12
how 9
different 6
actually 7
that. 11
know, 13
was 25
very 12
his 13
He 10
business 10
he 32
everything 6
out 13
don't 10
That's 6
were 9
had 7
one 16
back 7
go 9
some 11
did 7
million 9
-- 29
been 11
we're 12
but 13
said 8
if 13
which 6
u

Let's now try to make that program a bit better! You can just edit it above and leave the final program.
1. Although it was useful to learn about `if` control structures – and we will use them a lot – you don't actually need to use one here. Do you instead remember about the `get(<key>, <default>)` method on a dict that we saw earlier? Try using it. *(Warning: this can be challenging so it is fine to skip it at first and come back later)*.

2. The above code is hardcoded to print words that occur 2 or more times. This makes the list shorter by leaving out words that occur only once (the "hapax legomena" of the list — there are always a lot of these, often about 40% of the word types). But it's still a long list. We might want to only print words that occur 3 or more times, say. Make the minimum number of times for a word to occur to print it another parameter of the second function, and have the top-level code call it with the value 3.

3. At the other extreme, it might not be very interesting knowing how often the candidates say "the" or "to". Such function words often don't seem to carry much content. (Though, of course, be aware of the work of people such as [James Pennebaker](http://www.secretlifeofpronouns.com/), who emphasizes how much social meaning can be conveyed by function words. Lists of common function words that you are not going to count are conventionally called **stop words** in computational work. Modify the first program to also accept a list of stop words which you don't put in the hash. Modify the top-level code so that it doesn't count `['the', 'a', 'an', 'that', 'and', 'to', 'of']`

4. We're starting to suffer badly from simply tokenizing by dividing on whitespace. We could do a little better by simply "whiting out" the commonest punctuation marks that glom on to words. Before splitting on white space, we could change the string to delete letters like: `'.', ',', '"'`. Remember the `replace()` method on `str` that we saw last time. Look at your output, you may well want to delete a few more. Doing this will do a little textual damage; e.g., `30,000` will become `30000`, but it won't be too bad. You may want to not delete `'`, though, so that you don't damage words like `isn't`.

5. It might also be useful to lowercase all tokens, so that words don't become different just because they are at the start of a sentence. Of course, you'll then just have to be smart enough to recognize that `irs` means the `IRS`.

6. It would be good to also add up how many non-stop words were spoken by each candidate. Who spoke the most?

7. To normalize for frequency, it would be useful to also work out the percent of times the word each candidate says is a certain word. So, in the second function, also print the percent as well as the raw count.

8. Find at least one interesting difference in word use between the two candidates, and put it in the cell below!

In [21]:
# leaving the old code above and writing new code here

def good_wordcounter(file_name, stop_words):
    text_file = open(file_name) 
    word_count = {}

    for line in text_file:
        sentence = line.strip()
        
        # Q4 fix 
        # come back to this after the regex exercises, could this be done better?
        punc = ['.', ',', '"', "?", "–"]
        for i in punc:
            sentence = sentence.replace(i, " ")
        
        clean_sent = sentence.split()
        
        for word in clean_sent:
            # Q5 fix
            word = word.lower()
            # Q3 fix
            if word not in stop_words:
                #Q1 fix – in your own words, explain how and why this works
                word_count[word] =  word_count.get(word, 0) + 1
    
    text_file.close()
    
    return word_count

def good_wordcount_printer(word_count_dict, frequency): 
    # Q6 fix 
    total = 0
    #Q2 fix – notice change to freq, why do you think that is?
    for (word, freq) in word_count_dict.items(): 
        total = total + freq
    
    # notice that this line setting up the loop is the same as the previous one - why not combine the two into one? 
    for (word, freq) in word_count_dict.items(): 
        if freq >= frequency:
            # Q7 fix
            print(word, freq, freq * 100 / total)
    
    return total

            
            
# using the new functions
files = ['debate-clinton.txt', 'debate-trump.txt']
stop_words = ['the', 'a', 'an', 'that', 'and', 'to', 'of']

for file in files:
    count = good_wordcounter(file, stop_words)
    
    print("-----------------")
    print(f"currently printing the word count of file: {file}")
    print("-----------------")

    total_count = good_wordcount_printer(count, 3)
    
    print(f"the total number of non-stop words spoken: {total_count}")
    

-----------------
currently printing the word count of file: debate-clinton.txt
-----------------
how 13 0.2729372244383792
are 38 0.797816502204493
you 71 1.4906571488557632
donald 21 0.4408985933235356
well 35 0.7348309888725593
lester 6 0.1259710266638673
for 42 0.8817971866470712
us 20 0.419903422212891
question 4 0.08398068444257821
in 96 2.015536426621877
this 32 0.6718454755406257
is 56 1.175729582196095
really 19 0.3989082511022465
what 39 0.8188116733151375
kind 10 0.2099517111064455
country 16 0.33592273777031284
we 128 2.6873819021625027
want 20 0.419903422212891
be 57 1.1967247533067396
future 5 0.10497585555322275
we'll 6 0.1259710266638673
build 3 0.06298551333193365
together 6 0.1259710266638673
my 10 0.2099517111064455
second 4 0.08398068444257821
so 40 0.839806844425782
i 134 2.81335292882637
think 39 0.8188116733151375
about 34 0.7138358177619147
lot 18 0.37791307999160195
first 6 0.1259710266638673
have 84 1.7635943732941424
economy 13 0.2729372244383792
everyone 6 0

### Bonus: Getting word counts from the Google Books data

The raw data files for the Google Books collection are available for 
download. The files are huge, so I created a tiny sample in the file `googlebooks-eng-all-1gram-20120701-a-sample`.

The format of this file is as follows (whitespace inserted for 
readability):

```
word TAB year TAB match_count TAB volume_count NEWLINE
```

The TAB character is "\t", which you can treat like any other (for
example, you can split a string on "\t"). The `match_count` is how many times the word occurred and the `volume_count` is a smaller number for how many _different_ books it occurred in. We will use the `match_count`. Note that the words have also been disambiguated by part of speech where ambiguous. We'll get to that later.

Your first task: complete googlebooks_counts_by_year so that it processes
my sample file and returns a 2d dictionary (a top-level dictionary whose values are each a dictionary!) with this structure:

{
  word1: {year1: count, year2: count ...},
  word2: {year1: count, year2: count ...},
  ...
}

where the contents of the year dicts is determined by the file.
(That is, different words will have different years and counts
associated with them.)

In [22]:
def googlebooks_counts_by_year(filename):
    """Maps a Google books 1-grams file to a 2d dictionary
    giving each word's counts by year.
    Print statements (commented out) added to aid understanding. """
    # Initialize the top level dict
    all_counts = {}

    # open the file for reading
    gcountsfile = open(filename)

    # go through the open file line by line
    for line in gcountsfile:        
        # get rid of the line break at the end
        line = line.strip()
        #print(line)

        # split sentence at tabs
        fields = line.split('\t')
        #print(fields)
        
        word = fields[0]
        #print(word)
        
        year = fields[1]
        #print(year)
        
        count = int(fields[2])
        #print(count)
        
        if (word not in all_counts):
            # Initialize a new empty dict for that word type
            all_counts[word] = {}
            
        # Now we know that word is in the all_counts dict!
        all_counts[word][year] = count
        
    # close the file after reading
    gcountsfile.close()
    
    # return the dict
    return all_counts


Second task: Complete the function googlebooks_year_collapse so that it takes as input the output of googlebooks_counts_by_year and collapses
it down so that each word is associated with its single tokencount
for the full, obtained by summing up all of the counts for the
years associated with that word. 

In [23]:
def googlebooks_year_collapse(d):
    """Convert the output of googlebooks_counts_by_year to 
    a simpler dict mapping words to counts."""
    collapsed = {}
    
    # taking every word in the 2d dictionary from before
    for word in d.keys():
        # making a counter
        total = 0
        
        # getting all the years + counts for each word
        year_dict = d[word]
        for (year,count) in year_dict.items():
            # adding the count for currret year to total to get count across all years 
            total = total + count
        # saving the total count for the word in the new dict
        collapsed[word] = total
        
    return collapsed


Write something at the top-level to call this code on the file `googlebooks-eng-all-1gram-20120701-a-sample`:

In [24]:
details_dict = googlebooks_counts_by_year(
    'googlebooks-eng-all-1gram-20120701-a-sample')
collapsed_dict = googlebooks_year_collapse(details_dict)

print('Collapsed counts')
print('================')
for key in collapsed_dict.keys():
    print(key, collapsed_dict[key])

print()
print('All counts')
print('================')
for key in details_dict.keys():
    print(key, details_dict[key])

Collapsed counts
abstain_VERB 1022507
abstained 459896
abstentions.13 75
abstenue_NOUN 71
abstersion_ADP 93
abstinebat_X 72
abstinentes_X 88
abstinentiae_DET 122
abstinuere 79
abstraction3 66
abstractions.11 51
abstrahat 228
abstraits_VERB 164
abstrusen 86
abstulissent 77
absurd.14_ADP 74
abtsract 59
abuse.53_NOUN 59
abutere_VERB 168
abwälzen_X 60
abzuschreiben 275
abzustatten_NOUN 54
acac_NOUN 868
academies.4_NOUN 71
acanthopores_NOUN 4012
acarne 457
acaulay_NOUN 201
acccompanying_VERB 197
accelerandi_VERB 49
acceleratively 344
accentual_ADJ 79292
accentual_VERB 165
accept.63 88
acceptance.13 361
acceptance.1_VERB 69
acceptando 171
accepted.25_NOUN 44
accepto_NOUN 170
accesorio_NOUN 116
accessio_ADP 84
accessories 1419295
acci_X 1100
accidants_NOUN 48
accidente_ 46
accipis_X 506
acclamatisation 47
accmnulated_VERB 43
accoidingto 47
accommodatethe 79
accompaniedbya 56
accompanyed_ADJ 76
accompanying1 119
accordina_NOUN 256
accordingly.82 152
accordingly.85_ADP 46
accordingly.94 75
acco

## Part 2: Regular Expressions in Python

Regular expressions – regex – are super useful when processing text in Python! However, there are so many different patterns and ways to use the re module that it is impossible to learn by heart. So instead, this part of the HW is for you to play around and get comfortable with using regex. Also, head to [regex101.com](http://www.regex101.com/) for a nice place to test out your regex patterns before running them here.  

In [25]:
p = re.compile('[a-z]+')
p

re.compile(r'[a-z]+', re.UNICODE)

In [26]:

print(p.match("å"))

None


In [27]:
# try a few different things instead of 'tempo' - can you find the things that don't match?
m = p.match('tempo')
m

<re.Match object; span=(0, 5), match='tempo'>

In [28]:
m.group()

'tempo'

In [29]:
m.start(), m.end()

(0, 5)

In [30]:
print(p.match('::: message'))

None


In [31]:
m = p.search('::: message'); print(m)  

<re.Match object; span=(4, 11), match='message'>


In [32]:
p = re.compile('[a-z]+')
m = p.search( '2942a9vv4dxaq42' )
if m:
    print('Match found: ', m.group())
else:
    print('No match')

Match found:  a


In [33]:
p.findall('2942a9vv4dxaq42')

['a', 'vv', 'dxaq']

In [34]:
# Shortcut pattern and matcher in one!
re.match(r'From\s+', 'From amk Thu May 14 19:12:10 1998').group()

'From '

### Groups

In [35]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
m.group(0)

'abcd'

In [36]:
m.group(1)

'abc'

In [37]:
p = re.compile(r'(\d+)\s+(\w+)\s+St')
m = p.search('I live at 1345 Cowper St')
m.group(0)

'1345 Cowper St'

In [38]:
m.group(1)

'1345'

In [39]:
m.group(2)

'Cowper'

In [40]:
m.group(2)

'Cowper'

In [41]:
p = re.compile(r'(\d+)\s+(\w+)\s+St')
m = p.search('I live at 1345 Cowper St')
m.group(0)

'1345 Cowper St'

In [42]:
m.group(1)

'1345'

In [43]:
m.group(2)

'Cowper'

### Substitutions in a string

In [44]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

### Splitting on a regular expression

In [45]:
# A slightly better word tokenizer
# The re split method returns you things that MATCH
# the regular expression and skips stuff in between
s = '''\'As wet as ever,\' said Alice
    in a melancholy tone: \'it doesn\'t seem to
    dry me at all.\''''
p = re.compile('\W+')
p.split(s)

['',
 'As',
 'wet',
 'as',
 'ever',
 'said',
 'Alice',
 'in',
 'a',
 'melancholy',
 'tone',
 'it',
 'doesn',
 't',
 'seem',
 'to',
 'dry',
 'me',
 'at',
 'all',
 '']

In [46]:
# Again you can shortcut this.
re.split('\W+', s)

['',
 'As',
 'wet',
 'as',
 'ever',
 'said',
 'Alice',
 'in',
 'a',
 'melancholy',
 'tone',
 'it',
 'doesn',
 't',
 'seem',
 'to',
 'dry',
 'me',
 'at',
 'all',
 '']

Finally, I might note that there's even more complex and sometimes useful stuff you can do with regex that hasn't yet been covered. You can find all the glorious and messy details in the Python 3 library documentation: Case insensitivity, non-capturing groups, ....

## Assignment

In [47]:
tweets = (
    """@Becky17 - i'm having a little trouble "getting"<br /> the whole twibes thing (but sometimes u gotta just get in there and try it).  :-)""",
    """Oh .. and follow @Spyker3292, @Domness, @Karlkempobrien, @Duidl_Media and @Chasetastic. Cheers for the Congrats! :D | #FollowSaturday""",
    """blade--trinity;;; sweeeeeeet. :)""",
    """@renay Thanks Renay! $9,000 yay =)""",
    """@denvy can try :) drop a tweet with "##awaresg_tshirts" so i can <strong>track</strong> orders #awaresg""",
    """U need to chk out & follow here, a more beautiful animal not anywhere else! @EmmaRileySutton :) #followfriday""",
    """@LadyB84 Manchester United??? Really??? Breaks my heart :-(http://www.twitpic.com/4x1fn""",
    """Can't wait till tomorrow =D""",
    """Big Shot's Funeral » Google » Peoria making its case for Google ... http://cli.gs/Wa8za#heading1.""",
    """Contact email@address.org today""",
    """@linguist278: Variations on phone numbers: +1 (800) 123-4567, (800) 123-4567. Not a real tweet!""",
    """RT @StanfordPraglab: Mole Day is coming up. Theme is Animole Kingdom: http://en.wikipedia.org/wiki/Mole_Day #Holidays :-)"""
    )


Write a function that takes a list of strings, texts, and a regular expression, regex, as input and prints to standard output the subset that match regex.

In [48]:
def matcher(texts, regex):
    """Takes a list of strings texts as input and prints to standard output the subset that match regex."""
    p = re.compile(regex)
    for str in texts:
        if p.match(str):
            print(str)


Use this function to test out writing a few regular expressions, testing on the data above.

In [49]:
def contains_hashtag(texts):
    """Uses matcher to find tweets that contain a hashtag. Assume a hashtag begins with # and has a non-null sequence of non-space characters after it."""
    # If we just literally use the definition given in the docstring
    # then the regex could be '.*#\S+.*' but it will match the # that is
    # part of a URL (a # followed by a string in a URL specifies going to
    # that target on a page). We exclude that by requiring nothing or
    # a non-letter character before
    matcher(texts, '(|.*\W)#\S+.*')


In [50]:
def contains_money(texts):
    """Uses matcher to find tweets that contain a money amount. Assume a money amount begins with $ and has a non-null sequence of digits and periods after it."""
    matcher(texts, '.*\$(\d|\.)+.*')

Write a function that takes a list of strings, texts, and a regular expression, regex, as input and prints to standard output the substrings of each string that match regex.

In [51]:
# This version will only find the first match when there are several
def searcher2(texts, regex):
    """Takes a list of strings texts as input and prints to standard output the substrings of each that match regex."""
    p = re.compile(regex)
    # To find ALL the matches, you need to iterate with a while loop!
    # However, there weren't any examples that had multiple smileys 
    for str in texts:
        m = p.search(str)
        if m:
            print(m.group())

In [52]:
def searcher(texts, regex):
    """Takes a list of strings texts as input and prints to standard output the substrings of each that match regex."""
    p = re.compile(regex)
    # To find ALL the matches, you need to iterate with a while loop!
    # However, there weren't any examples that had multiple smileys 
    for str in texts:
        start_pos = 0
        while True:
            m = p.search(str, start_pos)
            if m:
                print(m.group())
                start_pos = m.end()
            else:
                break;

Use this function to test out writing a few regular expressions, testing on the data above.

In [53]:
def smileys(texts):
    """Uses searcher to find smiley faces, such as :) that appear in the list of strings, texts."""
    pass

In [54]:
smileys(tweets)

In [55]:
example_text = ["Yes. :) I'm really happy. :D Except when I'm sad. :-("]
smileys(example_text)

Use the included file words-english.txt and search it 
for words that have a consonant cluster of 4 or more consonants at the end. We're just using a count of four orthographic consonants, not sounds (phonemes). We won't count y since it is usally a vowel at the end of words.

Complete the function below

In [56]:
def final_consonant_clusters(filename):
    consonants = "bcdfghjklmnpqrstvwxz" # 'y' left out for added interest
    pattern = '[' + consonants + ']{4}$'
    # print(pattern)
    p = re.compile(pattern)
    answer = []
    file = open(filename)
    for line in file:
        # For this to work, you have to either strip the \n off the line
        # or to make the match more general by putting something like 
        # \s*$ at the end.
        line = line.strip()
        m = p.search(line)
        if m:
            answer = answer + [line]
    file.close()
    return answer

In [57]:
final_consonant_clusters('words-english.txt')

['amongst',
 'angst',
 'arclength',
 'eighth',
 'Ernst',
 'Hirsch',
 'length',
 'Messrs',
 'strength',
 'thousandth',
 'twelfth',
 'unbeknownst',
 'warmth',
 'wavelength']

Use the included file gaddafi.txt and write a regular expression to match instances of his surname. You should match the first 112 but not the last 8!

In [58]:
def gaddafi_matches(filename):
    pre_last = '([AEae][lI][- ])?'
    main_last = "[KQG]h?(a|ua|e|u)([dt]h?(d|h|'|[td]h)?|zz)aff?[iy]"
    last = pre_last + main_last
    first = "(M(o|u|ou)(['`h]?[aA])?|O|Mau)mm?[ea]r?"
    forward = first + ' (\w+ )*' + last + '( |$)'
    backward = last + ', ' + first
    pattern = forward + '|' + backward
    p = re.compile(pattern)
    matches = 0
    file = open(filename)
    for line in file:
        line = line.strip()
        m = p.search(line)
        if m:
            # print(line)
            matches = matches + 1
        else:
            print('Didn\'t match: ' + line)
    file.close()
    print('=== There were ' + str(matches) + ' matches. ===')

In [59]:
gaddafi_matches('gaddafi.txt')

Didn't match: # I think you shouldn't find these ones
Didn't match: Gagafy
Didn't match: Mummar Gadddafi
Didn't match: Quudafi, Muammar
Didn't match: Qudai, Muammar
Didn't match: Quhafi, Muammar
Didn't match: Mummar Gaddafiy
Didn't match: Mummar Gadaf
Didn't match: Omar Qaafi
=== There were 112 matches. ===
